# Converting Data in XPT format to CSV format

In [3]:
import xport
import csv
import os
from tqdm import tqdm

## Method for Converting an XPT File to CSV
This function converts an XPT file to a CSV file, allowing for column renaming and handling missing columns gracefully.

In [4]:
def convert_xpt_to_csv(columns_to_save, new_columns, xpt_file_path, csv_path_to_save):
    """
    Converts an XPT (SAS Transport) file to a CSV file, renaming selected columns and handling missing columns gracefully.
    
    Parameters:
        columns_to_save (list): List of original column names to extract.
        new_columns (list): List of new column names to rename extracted columns.
        xpt_file_path (str): Path to the input XPT file.
        csv_path_to_save (str): Path to save the output CSV file.
    """
    try:
        column_mapping = dict(zip(columns_to_save, new_columns))
        
        with open(xpt_file_path, 'rb') as xpt_file:
            reader = xport.XportReader(xpt_file)
            available_fields = set(reader.fields)
            
            missing_columns = [col for col in columns_to_save if col not in available_fields]
            if missing_columns:
                print(f"Warning: Missing columns in XPT file: {missing_columns}")

            os.makedirs(os.path.dirname(csv_path_to_save), exist_ok=True)

            with open(csv_path_to_save, 'w', newline='') as out_csv:
                writer = csv.DictWriter(out_csv, fieldnames=new_columns)
                writer.writeheader()

                for row in reader:
                    writer.writerow({
                        new_name: getattr(row, old_name, "nan") if old_name in available_fields else "nan"
                        for old_name, new_name in column_mapping.items()
                    })

        print(f"Data successfully saved to {csv_path_to_save}")

    except Exception as e:
        print(f"Error: {e}")

## Column Mappings for XPT to CSV Conversion
This section defines the selected columns from various XPT datasets and their corresponding renamed headers for a structured CSV output.

In [5]:
# Body Measurements (BM) Data
columns_to_save_bm = ['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXCALF', 'BMXARML', 'BMXARMC', 'BMXWAIST', 'BMXTHICR', 'BMAAMP']
new_columns_bm = ["id", "weight", "height", "bmi", "upper_leg_length", "maximal_calf_circumference", "upper_arm_length", "arm_circumference", "waist_circumference", "thigh_circumference", "amputation"]

# Demographics (DEMO) Data
columns_to_save_demo = ['SEQN', 'RIAGENDR', 'RIDAGEEX', 'RIDRETH1', 'RIDEXPRG']
new_columns_demo = ["id", "gender", "age_in_months", "ethnicity", "pregnancy_status"]

# Lean Mass Data
columns_to_save_lean_mass_data = ['SEQN', 'DXAEXSTS', 'DXDTOFAT', 'DXDTOLI']
new_columns_lean_mass_data = ["id", "exam_status", "total_fat_mass", "total_lean_mass"]

# Cardiovascular (CV) Data
columns_to_save_cv = ['SEQN', 'CVDFITLV']
new_columns_cv = ["id", "cv_fitness_level"]

## Converting XPT Files to CSV Across Multiple Years (1999-2006)
This section processes multiple NHANES XPT files from different survey years (1999–2006), converting them to CSV format with proper column mapping.

In [8]:
survey_cycles = [
    ("1999_2000", "BMX", "body_measures", columns_to_save_bm, new_columns_bm),
    ("1999_2000", "DEMO", "demographic_data", columns_to_save_demo, new_columns_demo),
    ("1999_2000", "dxx", "lean_fat_data", columns_to_save_lean_mass_data, new_columns_lean_mass_data),
    ("1999_2000", "CVX", "cv_level", columns_to_save_cv, new_columns_cv),

    ("2001_2002", "BMX_B", "body_measures", columns_to_save_bm, new_columns_bm),
    ("2001_2002", "DEMO_B", "demographic_data", columns_to_save_demo, new_columns_demo),
    ("2001_2002", "dxx_b", "lean_fat_data", columns_to_save_lean_mass_data, new_columns_lean_mass_data),
    ("2001_2002", "CVX_B", "cv_level", columns_to_save_cv, new_columns_cv),  

    ("2003_2004", "BMX_C", "body_measures", columns_to_save_bm, new_columns_bm),
    ("2003_2004", "DEMO_C", "demographic_data", columns_to_save_demo, new_columns_demo),
    ("2003_2004", "dxx_c", "lean_fat_data", columns_to_save_lean_mass_data, new_columns_lean_mass_data),
    ("2003_2004", "CVX_C", "cv_level", columns_to_save_cv, new_columns_cv),

    ("2005_2006", "BMX_D", "body_measures", columns_to_save_bm, new_columns_bm),
    ("2005_2006", "DEMO_D", "demographic_data", columns_to_save_demo, new_columns_demo),
    ("2005_2006", "dxx_d", "lean_fat_data", columns_to_save_lean_mass_data, new_columns_lean_mass_data),
]


print("Processing XPT files...")
for cycle, xpt_file, csv_file, columns, new_columns in tqdm(survey_cycles, desc="Converting", unit="file"):
    xpt_file_path = f"xpt/{cycle}/{xpt_file}.xpt"
    csv_path_to_save = f"csv/{cycle}/{csv_file}.csv"

    convert_xpt_to_csv(columns, new_columns, xpt_file_path, csv_path_to_save)

print("All files processed successfully.")

Processing XPT files...


Converting:   7%|████▌                                                                | 1/15 [00:03<00:43,  3.12s/file]

Data successfully saved to csv/1999_2000/body_measures.csv


Converting:  13%|█████████▏                                                           | 2/15 [00:12<01:30,  6.94s/file]

Data successfully saved to csv/1999_2000/demographic_data.csv


...

Data successfully saved to csv/2005_2006/body_measures.csv


Converting:  93%|███████████████████████████████████████████████████████████████▍    | 14/15 [01:30<00:04,  4.69s/file]

Data successfully saved to csv/2005_2006/demographic_data.csv


Converting: 100%|████████████████████████████████████████████████████████████████████| 15/15 [01:45<00:00,  7.05s/file]

Data successfully saved to csv/2005_2006/lean_fat_data.csv
All files processed successfully.
